In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import random
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')

In [2]:
# load data
train_set = pd.DataFrame.from_csv('sqf_train_cpw.csv', index_col = False)
test_set = pd.DataFrame.from_csv('sqf_test_cpw.csv', index_col = False)


In [14]:
# Explore the data a bit
# feature names and number of rows/cols
# print train_set.columns
# print train_set.shape

# distribution of various features in the training set
# print train_set['year'].value_counts()
# print train_set['found.weapon'].value_counts()
# print train_set['precinct'].value_counts()
# print train_set['suspect.race'].value_counts()

# proportion of features in the training set                         
train_set['suspect.race'].value_counts() / np.sum(train_set['suspect.race'].value_counts())
train_set['suspect.sex'].value_counts() / np.sum(train_set['suspect.sex'].value_counts())

# Warmup question:
# which precinct in the training set has the highest percentage of successful stops, i.e. stops where found.weapon==True?

male      0.965389
female    0.034611
Name: suspect.sex, dtype: float64

In [20]:
# join and re-split data to get all categories
train_set['set'] = 'train'
test_set['set'] = 'test'
joined_data = train_set.append(test_set)

# select all non-real-valued columns (besides 'set' and 'id') and convert to one-hot encoding
col_names = joined_data.columns
col_names = col_names.difference(['id', 'set', 'suspect.age', 'suspect.weight', 'suspect.height', 'observation.period'])
joined_data = pd.get_dummies(data=joined_data, columns=col_names, sparse=True)


In [22]:
col_names

Index([u'additional.associating', u'additional.direction',
       u'additional.evasive', u'additional.highcrime',
       u'additional.investigation', u'additional.other',
       u'additional.proximity', u'additional.report', u'additional.sights',
       u'additional.time', u'arrested', u'day', u'found.gun', u'found.weapon',
       u'inside.outside', u'location.housing', u'month', u'officer.uniform',
       u'precinct', u'radio.run', u'stopped.bc.bulge', u'stopped.bc.casing',
       u'stopped.bc.clothing', u'stopped.bc.desc', u'stopped.bc.drugs',
       u'stopped.bc.furtive', u'stopped.bc.lookout', u'stopped.bc.object',
       u'stopped.bc.other', u'stopped.bc.violent', u'suspect.build',
       u'suspect.race', u'suspect.sex', u'time.period', u'year'],
      dtype='object')

In [18]:
# what does the data look like after converting to one-hot encoding?
list(joined_data.columns)

['id',
 'suspect.age',
 'suspect.height',
 'suspect.weight',
 'observation.period',
 'set',
 'additional.associating_False',
 'additional.associating_True',
 'additional.direction_False',
 'additional.direction_True',
 'additional.evasive_False',
 'additional.evasive_True',
 'additional.highcrime_False',
 'additional.highcrime_True',
 'additional.investigation_False',
 'additional.investigation_True',
 'additional.other_False',
 'additional.other_True',
 'additional.proximity_False',
 'additional.proximity_True',
 'additional.report_False',
 'additional.report_True',
 'additional.sights_False',
 'additional.sights_True',
 'additional.time_False',
 'additional.time_True',
 'arrested_False',
 'arrested_True',
 'day_Friday',
 'day_Monday',
 'day_Saturday',
 'day_Sunday',
 'day_Thursday',
 'day_Tuesday',
 'day_Wednesday',
 'found.gun_False',
 'found.gun_True',
 'found.weapon_False',
 'found.weapon_True',
 'inside.outside_False',
 'inside.outside_True',
 'location.housing_housing',
 'locati

In [23]:
# remove redundant columns (binary columns of the form 'variable_False')
redundant_cols = []
for name in list(joined_data):
    if "False" in name:
        redundant_cols.append(name)
joined_data.drop(redundant_cols, inplace=True, axis=1)

# verify that redundant columns have been removed
#list(joined_data.columns)

In [26]:
# split data again
train = joined_data.loc[joined_data['set'] == 'train']
train = train.drop(['set'], axis=1)
test = joined_data.loc[joined_data['set'] == 'test']
test = test.drop(['set'], axis=1)

In [31]:
# split training data into features and outcome (numpy arrays, to feed to sklearn algorithms)
label_train = np.ravel(train[['found.weapon_True']].values)
pred_train = train.drop(['id', 'arrested_True', 'found.weapon_True', 'found.gun_True'], axis=1)

# print pred_train.head()
pred_train = pred_train.values 


In [41]:
# format test data
results = test.copy()
label_test = np.ravel(test[['found.weapon_True']].values)
pred_test = test.drop(['id', 'arrested_True', 'found.weapon_True', 'found.gun_True'], axis=1)
feature_names = list(pred_test.columns.values)
pred_test = pred_test.values 

In [28]:
# train the model with 1000 trees, 4 parallel processes, and 10 min samples to split a node 
num_trees = 200
rf = RandomForestClassifier(n_estimators=num_trees, n_jobs=2, min_samples_split=10, verbose=2, oob_score = True)
rf.fit(X=pred_train, y=label_train)

# generate predictions and add them to 'results'
rf_predictions = rf.predict_proba(pred_test)[:, 1]
results['preds'] = rf_predictions

# get AUC score (produce probabilistic predictions)
print roc_auc_score(label_test, rf_predictions)

# get accuracy (predict the class)
rf_predictions_class = rf.predict(pred_test)
print accuracy_score(label_test, rf_predictions_class, normalize=True)

# Questions:
# What happens to AUC if I change the target variable to found.gun_True?
# What happens to AUC and accuracy if I forgot to take out found.weapon_True from the features?  Why?
# How does AUC change if I forgot to remove 'arrested_True'?  Does that mean I should remove it?
# What are the most five important features?

[Parallel(n_jobs=2)]: Done   1 out of 200 | elapsed:    3.1s remaining: 10.3min
[Parallel(n_jobs=2)]: Done 101 out of 200 | elapsed:  1.8min remaining:  1.7min
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:  3.6min finished
[Parallel(n_jobs=2)]: Done   1 out of 200 | elapsed:    0.1s remaining:   22.0s
[Parallel(n_jobs=2)]: Done 101 out of 200 | elapsed:    5.2s remaining:    5.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:   10.2s finished


building tree 1 of 200
building tree 2 of 200
building tree 3 of 200
building tree 4 of 200
building tree 5 of 200
building tree 6 of 200
building tree 7 of 200
building tree 8 of 200
building tree 9 of 200
building tree 10 of 200
building tree 11 of 200
building tree 12 of 200
building tree 13 of 200
building tree 14 of 200
building tree 15 of 200
building tree 16 of 200
building tree 17 of 200
building tree 18 of 200
building tree 19 of 200
building tree 20 of 200
building tree 21 of 200
building tree 22 of 200
building tree 23 of 200
building tree 24 of 200
building tree 25 of 200
building tree 26 of 200
building tree 27 of 200
building tree 28 of 200
building tree 29 of 200
building tree 30 of 200
building tree 31 of 200
building tree 32 of 200
building tree 33 of 200
building tree 34 of 200
building tree 35 of 200
building tree 36 of 200
building tree 37 of 200
building tree 38 of 200
building tree 39 of 200
building tree 40 of 200
building tree 41 of 200
building tree 42 of 200
b

[Parallel(n_jobs=2)]: Done   1 out of 200 | elapsed:    0.1s remaining:   26.7s
[Parallel(n_jobs=2)]: Done 101 out of 200 | elapsed:    7.2s remaining:    7.1s
[Parallel(n_jobs=2)]: Done 200 out of 200 | elapsed:   13.8s finished



0.966555488481


In [50]:
# Plotting question:
# Make a recovery plot: if you used the RF model to rank stops by model-predicted likelihood of weapon recovery, 
# from highest to lowest, what percent of weapons would you recover if you made the best x percent of stops?
# The plot should have percent of stops on the x axis and percent weapons recovered on the y axis





# HINTS:
# 1) order results by column 'preds'
# 2) add a column to results which is the cumulative sum of found.weapon_True
# 3) use the above cumulative sum to make a column which shows percent weapons recovered
# 4) add a column which counts the stops
# 5) use the above stop count column to make a column which shows percent of all stops
# 6) restrict to just the columns from 3) and 5), downsample to maybe 1000 rows
# 7) sort everything in ascending order by the column from 5), then plot.


